# RAG 시스템 - 저장된 문서 확인 (Document Viewing)

이 노트북은 ChromaDB에 저장된 문서들을 확인하고 검색하는 방법을 설명합니다.

## 주요 기능
1. 벡터 스토어 연결
2. 저장된 문서 조회
3. 문서 내용 확인

## 사전 요구사항
- ChromaDB 서버가 실행 중이어야 합니다
- 이전 단계에서 저장된 문서가 있어야 합니다

## 1. 필요한 라이브러리 임포트 및 설정

문서 조회에 필요한 기본 설정을 진행합니다:
- Chroma: 벡터 데이터베이스 접근
- HuggingFaceEmbeddings: 임베딩 모델
- 기본 설정값 정의

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# ChromaDB 설정
PERSIST_DIR = "./data"
COLLECTION_NAME = "rag_test"

## 2. 임베딩 모델 설정

문서 검색을 위한 임베딩 모델을 설정합니다:
- nomic-embed-text-v1 모델 사용
- CPU 기반 처리
- 정규화된 임베딩 사용

In [ ]:
def get_embeddings():
    """한국어에 최적화된 임베딩 모델 생성"""
    return HuggingFaceEmbeddings(
        model_name="nomic-ai/nomic-embed-text-v1",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

## 3. 벡터 스토어 로드

ChromaDB에서 저장된 문서들을 로드하는 함수입니다:
- 지정된 컬렉션에 연결
- 임베딩 모델 초기화
- 에러 처리 포함

In [ ]:
def load_vector_store():
    """ChromaDB에서 벡터 스토어 로드"""
    try:
        embedding_model = get_embeddings()
        vector_store = Chroma(
            persist_directory=PERSIST_DIR,
            embedding_function=embedding_model,
            collection_name=COLLECTION_NAME
        )
        return vector_store
    except Exception as e:
        print(f"❌ 벡터 스토어 로드 실패: {str(e)}")
        return None

## 4. 저장된 문서 확인

벡터 스토어에 저장된 모든 문서를 조회하고 출력합니다:
- 최대 100개 문서 조회
- 각 문서의 내용을 깔끔하게 출력
- 구분선으로 문서 구분

In [ ]:
# 벡터 스토어 로드
vector_store = load_vector_store()

if vector_store:
    # 저장된 모든 문서 가져오기 (최대 100개)
    results = vector_store.similarity_search("", k=100)
    
    print(f"📚 총 {len(results)}개의 문서가 저장되어 있습니다.\n")
    
    # 각 문서 내용 출력
    for i, doc in enumerate(results, 1):
        print(f"문서 {i}:")
        print(f"{doc.page_content}\n")
        print("-" * 50 + "\n")